In [ ]:
import EmbeddedCV
import cv2
import numpy as np
import os
from scipy.ndimage import convolve
from matplotlib import pyplot as plt
from QOI_encode import QOI_encode
from QOI_decode import QOI_decode

# Test verifiers

In [ ]:
import numpy as np

def check_correctness(y_pred, y):
    diff = y_pred.astype(np.int32) - y.astype(np.int32)
    abs_diff = np.abs(diff)
    max_diff = np.max(abs_diff)
    avg_diff = np.mean(abs_diff)

    diff_unacceptable = np.where(abs_diff > 2)

    for idx in zip(*diff_unacceptable):
        print(idx)

    if y_pred.shape[2] == 1:
        gray = True
    else: 
        gray = False

    cv2.imwrite("pred.png", y_pred)
    cv2.imwrite("true.png", y)
    cv2.imwrite("diff.png", abs_diff)

    print("Max diff:", max_diff)
    print("Avg diff:", avg_diff)

    return True

# Naive implementations

In [ ]:
from functions import gaussian_blur, canny_edge, sobel_min_max, median_blur

# Tests

Setup

In [ ]:
channels = 3
rows = 576
cols = 768

cam = EmbeddedCV.CamSimulator(True, channels, rows, cols)
host = EmbeddedCV.Host(cam, channels, rows, cols)

Test cases

In [ ]:
image_path = "Background_Subtraction_Tutorial_frame_1.png"
image = cv2.imread(image_path)

flat_image = image.flatten()

In [ ]:
cam.StoreData(flat_image)

Sanity Check

In [ ]:
# host_output = np.zeros(shape=(channels*rows*cols), dtype=np.uint8)

# host.Sanity(host_output)
# host_output = host_output.reshape((rows, cols, channels))

# naive_output = image

# check_correctness(y_pred=host_output, y=naive_output)

MedianBlur (0 error when min max buffers whole image)

In [ ]:
# kernel_height, kernel_width = 3, 3

# host_output = np.zeros(shape=(channels*rows*cols), dtype=np.uint8)

# host.MedianBlur(host_output, kernel_height, kernel_width)
# host_output = host_output.reshape((rows, cols, channels))

# naive_output = median_blur(image, kernel_height, kernel_width)

# check_correctness(y_pred=host_output, y=naive_output)

GaussianBlur

In [ ]:
kernel_height, kernel_width = 5, 5
sigma_x, sigma_y = 0, 0

host_output = np.zeros(shape=(channels*rows*cols), dtype=np.uint8)

host.GaussianBlur(host_output, kernel_height, kernel_width, sigma_x, sigma_y)
host_output = host_output.reshape((rows, cols, channels))

naive_output = gaussian_blur(image, kernel_height, kernel_width, sigma_x, sigma_y)

check_correctness(y_pred=host_output, y=naive_output)

Sobel

In [ ]:
# host_output = np.zeros(shape=(channels*rows*cols), dtype=np.uint8)

# host.Sobel(host_output)
# host_output = host_output.reshape((rows, cols, channels))

# naive_output = sobel_min_max(image)

# check_correctness(y_pred=host_output, y=naive_output)

CannyEdge

In [ ]:
# lowThreshold = 100
# highThreshold = 200

# host_output = np.zeros(shape=(1*rows*cols), dtype=np.uint8)
 
# host.CannyEdge(host_output, lowThreshold, highThreshold)
# host_output = host_output.reshape((rows, cols, 1))

# naive_output = canny_edge(image, lowThreshold, highThreshold)

# check_correctness(y_pred=host_output, y=naive_output)

QOI

In [ ]:
# QOI_encode(flat_image, channels, rows, cols)

In [ ]:
# QOI_decode("img.bin")